In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import skew
from feature import is_outlier, simple_numerical, simple_categorical
%matplotlib inline 
plt.style.use('ggplot')

/home/jia/anaconda2/envs/py35/lib/python3.5/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
train_file="../data/train.csv"
test_file="../data/test.csv"
train_data = pd.read_csv(train_file)
test_data = pd.read_csv(test_file)

In [3]:
train_data['SalePrice']= np.log1p(train_data[train_data.columns.values[-1]])
train_num = train_data.select_dtypes(exclude=["object"]).iloc[:,1:-1]
test_num = test_data.select_dtypes(exclude=["object"]).iloc[:,1:]

simple_numerical(train_num)
simple_numerical(test_num)

train_cat = train_data.select_dtypes(include=["object"])
test_cat = test_data.select_dtypes(include=["object"])

train_cat = simple_categorical(train_cat)
train_cat = simple_categorical(test_cat)

X_train = train_num.join(train_cat)
y_train = train_data['SalePrice']

X_test = test_num.join(test_cat)

/home/jia/anaconda2/envs/py35/lib/python3.5/site-packages/pandas/core/indexing.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [4]:
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet,LassoCV,LassoLarsCV
from sklearn.model_selection import cross_val_score

def rmse_cv(model):
    rmse = np.sqrt(-cross_val_score(model, X_train, y_train, scoring = "neg_mean_squared_error", cv = 5))
    return rmse

In [6]:
model_ridge = Ridge()
alphas = [0.05, 0.1, 0.3, 1, 3, 5, 10, 15, 30, 50, 75]
cv_ridge = [rmse_cv(Ridge(alpha = alpha)).mean() 
            for alpha in alphas]

cv_ridge = pd.Series(cv_ridge, index = alphas)
cv_ridge.plot(title = "Validation - Just Do It")
plt.xlabel("alpha")
plt.ylabel("rmse")

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').